In [ ]:
import pandas as pd
import os
os.environ["R_HOME"] = "/Library/Frameworks/R.framework/Resources"
from pymer4.models import Lmer
import statsmodels.api as sm

In [ ]:
data_path = '/Users/jk1/Library/CloudStorage/OneDrive-unige.ch/icu_research/prehospital/intubation_trigger/data/Mastertabelle_filtered.xlsx'

In [ ]:
df = pd.read_excel(data_path)

In [ ]:
df = df[df.Comment != 'Reanimation']

In [ ]:
df.head()

In [ ]:
outcomes_of_interest = [
    "ICU [d]",
    "Ventilator [d]",
    "Hospital stay [d]",
    "28 d Survival [y/n]",
    "ITN ass. Pneumonia [y/n]"
    ]

In [ ]:
time_columns =["ICU [d]",
    "Ventilator [d]",
    "Hospital stay [d]"]
# in every column, if value ends with +, remove it
for col in time_columns:
    df[col] = pd.to_numeric(df[col].apply(lambda x: str(x).replace("+", '')), errors='coerce')


In [ ]:
df['Sverweis_Geschlecht_encoded'] = df['Sverweis_Geschlecht'].map({'männlich': 0, 'weiblich': 1})
df['28 d Survival [y/n]_encoded'] = df['28 d Survival [y/n]'].map({'n': 0, 'y': 1})
df['ITN ass. Pneumonia [y/n]_encoded'] = df['ITN ass. Pneumonia [y/n]'].map({'n': 0, 'y': 1})

In [ ]:
df.columns = df.columns.str.replace(' ', '_')

In [ ]:
# df['GCS_cat'] = pd.Categorical(df['GCS_(Erstbefund)'], categories=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ordered=True)
# # then convert to strings categories
# df['GCS_cat'] = df['GCS_cat'].astype(str)
# # and then convert to strings categories
# df['GCS_cat'] = pd.Categorical(df['GCS_cat'], categories=['3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15'], ordered=True)

In [ ]:
df['Einteilung_encoded'] = df['Einteilung_(Hauptdiagnose)'].map({'Krankheit': 0, 'Unfall': 1})

In [ ]:
# icu los
icu_los_df = df[["ICU_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
icu_los_df.columns = ['ICU_los', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {icu_los_df.isna().sum().sum()}')
icu_los_df.dropna(inplace=True)


overall_icu_los_model = Lmer(f"ICU_los ~ GCS + sex + age + (1|category)", data=icu_los_df, family='gaussian')
overall_icu_los_res = overall_icu_los_model.fit()
print(overall_icu_los_res)

In [ ]:
unadjusted_icu_los_model = sm.OLS(icu_los_df['ICU_los'], sm.add_constant(icu_los_df[['GCS']]))
unadjusted_icu_los_results = unadjusted_icu_los_model.fit()
print(unadjusted_icu_los_results.summary())

In [ ]:
# ventilator days
ventilator_days_df = df[["Ventilator_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
ventilator_days_df.columns = ['ventilator_days', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {ventilator_days_df.isna().sum().sum()}')
ventilator_days_df.dropna(inplace=True)

overall_ventilator_days_model = Lmer(f"ventilator_days ~ GCS + sex + age + (1|category)", data=ventilator_days_df, family='gaussian')
overall_ventilator_days_res = overall_ventilator_days_model.fit()
overall_ventilator_days_res

In [ ]:
unadjusted_ventilator_days_model = sm.OLS(ventilator_days_df['ventilator_days'], sm.add_constant(ventilator_days_df[['GCS']]))
unadjusted_ventilator_days_results = unadjusted_ventilator_days_model.fit()
print(unadjusted_ventilator_days_results.summary())

In [ ]:
# hospital stay
hospital_stay_df = df[["Hospital_stay_[d]", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
hospital_stay_df.columns = ['hospital_stay', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {hospital_stay_df.isna().sum().sum()}')
hospital_stay_df.dropna(inplace=True)

overall_hospital_stay_model = Lmer(f"hospital_stay ~ GCS + sex + age + (1|category)", data=hospital_stay_df, family='gaussian')
overall_hospital_stay_res = overall_hospital_stay_model.fit()
print(overall_hospital_stay_res)

In [ ]:
unadjusted_hospital_stay_model = sm.OLS(hospital_stay_df['hospital_stay'], sm.add_constant(hospital_stay_df[['GCS']]))
unadjusted_hospital_stay_results = unadjusted_hospital_stay_model.fit()
print(unadjusted_hospital_stay_results.summary())

In [ ]:
# 28 d Survival
survival_df = df[["28_d_Survival_[y/n]_encoded", 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
survival_df.columns = ['survival', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {survival_df.isna().sum().sum()}')
survival_df.dropna(inplace=True)

overall_survival_model = Lmer(f"survival ~ GCS + sex + age + (1|category)", data=survival_df, family='binomial')
overall_survival_res = overall_survival_model.fit()
print(overall_survival_res)
                    

In [ ]:
unadjusted_survival_model = sm.Logit(survival_df['survival'], sm.add_constant(survival_df[['GCS']]))
unadjusted_survival_results = unadjusted_survival_model.fit()
print(unadjusted_survival_results.summary())

In [ ]:
# pneumonia
pneumonia_df = df[['ITN_ass._Pneumonia_[y/n]_encoded', 'GCS_(Erstbefund)', 'Sverweis_Geschlecht_encoded', 'Alter__(Patient)', 'Einteilung_encoded']]
pneumonia_df.columns = ['pneumonia', 'GCS', 'sex', 'age', 'category']
print(f'number of nan (excluded): {pneumonia_df.isna().sum().sum()}')
pneumonia_df.dropna(inplace=True)

overall_pneumonia_model = Lmer(f"pneumonia ~ GCS + sex + age + (1|category)", data=pneumonia_df, family='binomial')
overall_pneumonia_res = overall_pneumonia_model.fit()
print(overall_pneumonia_res)

In [ ]:
unadjusted_pneumonia_model = sm.Logit(pneumonia_df['pneumonia'], sm.add_constant(pneumonia_df[['GCS']]))
unadjusted_pneumonia_results = unadjusted_pneumonia_model.fit()
print(unadjusted_pneumonia_results.summary())

In [ ]:
# save all model results in pdf with headings for each outcome and model

from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
import io

# Function to add model results to PDF
def add_model_results_to_pdf(c, model_results,heading, start_y=750):
    c.setFont("Helvetica", 12)
    y_position = start_y
    c.drawString(100, y_position, heading)
    y_position -= 20 
    
    text_object = c.beginText(40, y_position)
    text_object.setFont("Helvetica", 10)
    for line in model_results.split('\n'):
        text_object.textLine(line)
        y_position -= 10
        # Check if we need to start a new page
        if y_position < 40:  # Adjust based on your footer size
            c.drawText(text_object)
            c.showPage()  # Start a new page
            c.setFont("Helvetica", 12)
            y_position = start_y - 20  # Reset y_position for the new page
            text_object = c.beginText(40, y_position)
            text_object.setFont("Helvetica", 10)

    c.drawText(text_object)

# Create a new PDF with ReportLab
c = canvas.Canvas("/Users/jk1/Downloads/model_results.pdf", pagesize=letter)
height = letter[1]

# Assuming you have model results as strings, for example:
overall_icu_los_res_str = str(overall_icu_los_res)
unadjusted_icu_los_results_str = str(unadjusted_icu_los_results.summary())
# Add more model results strings as needed

add_model_results_to_pdf(c, overall_icu_los_res_str, "Adjusted ICU LOS Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, unadjusted_icu_los_results_str, "Unadjusted ICU LOS Model Results")
c.showPage()  # Start a new page for the next model if necessary


add_model_results_to_pdf(c, str(overall_ventilator_days_res), "Adjusted Ventilator Days Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_ventilator_days_results.summary()), "Unadjusted Ventilator Days Model Results")
c.showPage()  # Start a new page for the next model if necessary

add_model_results_to_pdf(c, str(overall_hospital_stay_res), "Adjusted Hospital Stay Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_hospital_stay_results.summary()), "Unadjusted Hospital Stay Model Results")
c.showPage()  # Start a new page for the next model if necessary

add_model_results_to_pdf(c, str(overall_survival_res), "Adjusted 28 d Survival Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_survival_results.summary()), "Unadjusted 28 d Survival Model Results")
c.showPage()  # Start a new page for the next model if necessary

add_model_results_to_pdf(c, str(overall_pneumonia_res), "Adjusted Pneumonia Model Results")
c.showPage()  # Start a new page for the next model if necessary
add_model_results_to_pdf(c, str(unadjusted_pneumonia_results.summary()), "Unadjusted Pneumonia Model Results")

# Save the PDF
c.save()
